# Lesson 4: Persistence and Streaming

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
import requests

In [2]:
WMO_CODES = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Foggy", 48: "Icy fog",
    51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
    61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
    71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
    80: "Slight showers", 81: "Moderate showers", 82: "Violent showers",
    95: "Thunderstorm", 99: "Thunderstorm with hail",
}

@tool
def get_current_weather(location: str) -> str:
    """Get the current weather for a given location."""
    # Step 1: geocode city name -> lat/lon
    geo = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": location, "count": 1},
        timeout=10,
    )
    geo.raise_for_status()
    results = geo.json().get("results")
    if not results:
        return f"Could not find location: {location}"
    place = results[0]
    lat, lon, name = place["latitude"], place["longitude"], place["name"]

    # Step 2: fetch current conditions
    weather = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": lat,
            "longitude": lon,
            "current": "temperature_2m,apparent_temperature,relative_humidity_2m,weather_code,wind_speed_10m",
            "temperature_unit": "fahrenheit",
            "wind_speed_unit": "mph",
            "timezone": "auto",
        },
        timeout=10,
    )
    weather.raise_for_status()
    c = weather.json()["current"]

    condition = WMO_CODES.get(c["weather_code"], f"Code {c['weather_code']}")
    return (
        f"Location: {name}\n"
        f"Temperature: {c['temperature_2m']}°F\n"
        f"Feels like: {c['apparent_temperature']}°F\n"
        f"Condition: {condition}\n"
        f"Humidity: {c['relative_humidity_2m']}%\n"
        f"Wind: {c['wind_speed_10m']} mph"
    )

tool = get_current_weather
print(type(tool))
print(tool.name)

<class 'langchain_core.tools.structured.StructuredTool'>
get_current_weather


In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [4]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [5]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            # Ollama sometimes wraps arg values as {'description': ..., 'value': ...}
            args = {
                k: v['value'] if isinstance(v, dict) and 'value' in v else v
                for k, v in t['args'].items()
            }
            result = self.tools[t['name']].invoke(args)
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [6]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOllama(model="lfm2.5-thinking", temperature=0.7)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [7]:
messages = [HumanMessage(content="What is the weather in seattle for today?")]

In [8]:
thread = {"configurable": {"thread_id": "1"}}

In [9]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-16T15:58:45.509941Z', 'done': True, 'done_reason': 'stop', 'total_duration': 646022292, 'load_duration': 36153084, 'prompt_eval_count': 149, 'prompt_eval_duration': 38153667, 'eval_count': 127, 'eval_duration': 553375375, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c672c-f279-7462-a1ca-4c6d73aac7ff-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Seattle'}, 'id': '23d3c4f2-c151-4739-bce8-2927fa88c87a', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 149, 'output_tokens': 127, 'total_tokens': 276})]
Calling: {'name': 'get_current_weather', 'args': {'location': 'Seattle'}, 'id': '23d3c4f2-c151-4739-bce8-2927fa88c87a', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='Location: Seattle\nTemperature: 33.3°F\nFeels like: 27.3°F\nCondition: Overcast\nHumidity: 94%\nW

In [10]:
messages = [HumanMessage(content="What about in la?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-16T15:58:51.862406Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1684480875, 'load_duration': 31213167, 'prompt_eval_count': 295, 'prompt_eval_duration': 32284125, 'eval_count': 355, 'eval_duration': 1559943697, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c672d-0740-73f0-969b-bac3d9ba6095-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Los Angeles'}, 'id': 'dccda08b-3eb4-48c4-90dd-da69744497e6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 295, 'output_tokens': 355, 'total_tokens': 650})]}
Calling: {'name': 'get_current_weather', 'args': {'location': 'Los Angeles'}, 'id': 'dccda08b-3eb4-48c4-90dd-da69744497e6', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='Location: Los Angeles\nTemperature: 54.2°F\nFeels like: 49.3

In [11]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="The current weather in **Los Angeles** (54.2°F) is warmer than in **Seattle** (33.3°F). Los Angeles is significantly warmer despite similar overcast conditions. Let me know if you'd like further details!", additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-16T15:58:57.36343Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1435962500, 'load_duration': 28638500, 'prompt_eval_count': 444, 'prompt_eval_duration': 32968250, 'eval_count': 289, 'eval_duration': 1273867793, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c672d-1db6-7c82-977e-d6c332b9a464-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 444, 'output_tokens': 289, 'total_tokens': 733})]}


In [12]:
messages = [HumanMessage(content="Which one would you recommend for a beginner first-time homebuyer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'lfm2.5-thinking', 'created_at': '2026-02-16T16:00:24.978111Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4844179208, 'load_duration': 36415708, 'prompt_eval_count': 519, 'prompt_eval_duration': 150888250, 'eval_count': 1008, 'eval_duration': 4449358513, 'logprobs': None, 'model_name': 'lfm2.5-thinking', 'model_provider': 'ollama'}, id='lc_run--019c672e-66a3-7112-a1f3-f02400fffc21-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'Seville'}, 'id': '0f373804-8df0-4af1-807d-70b052d64072', 'type': 'tool_call'}, {'name': 'get_current_weather', 'args': {'location': 'Los Angeles'}, 'id': 'fcb44f40-6e5c-48fc-85fc-7f1a438081cf', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 519, 'output_tokens': 1008, 'total_tokens': 1527})]}
Calling: {'name': 'get_current_weather', 'args': {'location': 'Seville'}, 'id': '0f373804-8df0-4af1-807d-70b052d64072', 't